# 1. PDF文件

PDF：Portable Document Format（便携式文档格式），由Adobe Systems于1993年发明，现已成为ISO 32000国际标准。

PDF应用场景：

- 学术出版：书籍出版物。
- 政府公文：中国税务局电子发票版式文件/各种发文。
- 企业文档：合同/报表/演示稿的跨平台保真传输。

解析PDF困难的根本原因：

| 难点维度         | 技术原理                                      | 现实影响案例                                  |
|------------------|---------------------------------------------|---------------------------------------------|
| **混合内容模型**  | 文本/图像/矢量图形混合存储                    | 同一行文字可能被拆分为多个不相连的文本块       |
| **无逻辑结构**   | 内容按渲染顺序存储而非逻辑顺序                | 表格数据可能被存储为绝对定位的文本片断         |
| **格式碎片化**   | 支持20+种编码方式（ASCIIHex/Base85/JPEG2000等） | 同一文档中不同页面可能采用完全不同的压缩编码    |
| **动态元素**     | 支持表格/注释/3D模型等交互元素                | 表单字段内容可能无法通过常规文本提取获取        |
| **历史兼容性**   | 需兼容1993年至今的所有版本特性                | 旧版Type 3字体与现代OpenType字体并存导致文本提取差异 |

商业级解决方案（99%+准确率），做不到100%。

# 2. PDF文件加载器

## 2.1 langchain中的PDF加载器

- PyPDF([PyPDFLoader](https://python.langchain.com/docs/integrations/document_loaders/pypdfloader)): 基于纯Python实现的PDF解析库。适用场景:快速提取PDF文本内容（无复杂格式要求）
- PDFPlumber([PDFPlumber](https://python.langchain.com/docs/integrations/document_loaders/pdfplumber)): 基于字符级的精确坐标定位实现的加载器，对中文比较友好。
- PyMuPDF([PyMuPDFLoader](https://python.langchain.com/docs/integrations/document_loaders/pymupdf/)): 一个轻量级 PDF 解析/渲染引擎。
- PyMuPDF4LLM([PyMuPDF4LLMLoader](https://python.langchain.com/docs/integrations/document_loaders/pymupdf4llm/)): 基于pymupdf进行封装，适配了 LLM 和 RAG 的场景。

https://python.langchain.com/docs/integrations/document_loaders/


# 3. PyMuPDF4LLMLoader

依赖于 pymupdf 和 pymupdf4llm 这两个库。

- pymupdf: 一个轻量级 PDF 和 XPS 解析/渲染引擎。提供全面的 PDF 操作功能：读取、编辑、提取内容、渲染等。
    - 由 Artifex Software 公司维护。
    - fitz: MuPDF 的原始渲染引擎名称。import fitz 
- pymupdf4llm: 基于pymupdf进行封装，适配了 LLM 和 RAG 的场景。

安装

```shell
pip install -U langchain-pymupdf4llm
```

参数：

- file_path: 文件路径
- headers: 默认为None,从网络路径下载文件的 GET 请求要使用的标头（此时 file_path 是一个web_url）
- password: str, 密码
- mode: 如何读取pdf文件，枚举值: single(整体读取)、page（分页读取）,默认是page
- pages_delimiter: str, 页面分隔符，设置该参数，会使用分隔符对文档进行分割。
- extract_images: bool，是否提取图片,默认为 False
- images_parser: 图片解析器，默认为None。可以是 OCR(RapidOCRBlobParser)/LLM图片解析器实例(LLMImageBlobParser)。
- table_strategy:
    - lines: 通过检测 PDF 中的线条来识别表格结构.（适用场景：传统网格状表格（有明显边框））
    - lines_strict：（默认）比 lines更严格的线条检测策略。要求表格有更完整、更规则的边框线。
    - text：基于文本对齐方式识别表格。不依赖线条，而是通过文本的空间排列关系。
    - explicit：需要用户明确指定表格边界线。通过 explicit_vertical_lines和 explicit_horizontal_lines参数提供线条位置。提供最高精度的控制。

参考文档:
https://python.langchain.com/docs/integrations/document_loaders/pymupdf4llm/

相关依赖包:

```python
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
from langchain_community.document_loaders.parsers import LLMImageBlobParser
from langchain_ollama import ChatOllama
```

In [1]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
from langchain_community.document_loaders.parsers import LLMImageBlobParser
from langchain_ollama import ChatOllama

docs_loader = PyMuPDF4LLMLoader(
    file_path="./pdf_1_5.pdf",
    extract_images=True,
    images_parser=LLMImageBlobParser(
        model=ChatOllama(model="qwen2.5vl:latest"),
        prompt="""你是一名助理，负责对图片进行摘要以便检索。
    “1. 这些摘要将被嵌入并用于检索原始图片。”
    “请提供图片的简洁摘要，并针对检索进行优化\n”
    “2. 从图片中提取所有文本。”
    “请勿从页面中排除任何内容。\n”
    “请将答案格式化为 Markdown 格式，不包含解释性文字”
    “并且开头不包含 Markdown 分隔符 ```。"""
    )
)

docs = docs_loader.load()
# Document(page_content="")
for doc_item in docs:
    print(doc_item.page_content)
    print("*" * 100)
    

![浦银国际研究 | 科技行业](#)





![科技行业 2025 年展望：AI 波潮

1. 展望 2025 年，AI 是主旋律，也是驱动力。生成式 AI 正在持续推动科技产业增量市场的高速增长，给科技行业带来增长动能。AI 服务器需求持续高增量，云端训练和推理的 AI 算力芯片处于高速成长的初期阶段。AI 从云侧向端侧渗透，拉动智能手机、个人电脑等消费电子终端的需求，反哺云端需求。本轮 AI 波潮是科技行业重大推动力。预计 2024 年科技行业的成长动能主旋律有望向 2025 年延续。建议关注 AI 算力芯片最受益玩家台积电和英伟达。

2. 生成式 AI 向端侧渗透，带动智能手机等消费电子行业成长。根据 Counterpoint 的预测，2024 年和 2025 年的 AI 智能手机渗透率分别将达到 18% 和 29%。预计 2025 年全球智能手机出货量将达到 12.4 亿部，同比增长 2%。这给予智能手机供应链较好的成长支撑，包括数量的增长以及高端项目占比提升。建议关注受益公司小米、比亚迪电子、立讯精密、蓝思科技等。

3. AI 是半导体行业最大增量，直接或间接带来晶圆代工和功率半导体的增量需求。预计 2024 年全球半导体行业规模将同比增长 16%，2025 年将同比增长 12%。中国晶圆代工行业仍将保持增长，主要代工厂的出货量将保持较好成长。建议关注华虹半导体，其市净率估值依然具备吸引力。建议关注受益于汽车电子和 AI 服务器相关的功率器件增量的标的，如新洁能、扬杰科技等。

4. 科技硬件估值合理，建议布局增加仓位。当前 A 股电子行业和 A 股半导体行业市盈率分别为 55.0x 和 89.0x，分别处于历史 75% 和 62% 的百分位。我们拟合的中国晶圆代工和中国功率半导体行业市盈率分别为 53.3x 和 64.5x，处于 94% 和 44% 的历史百分位。纳斯达克指数估值为 28.1x，处于 82% 的历史百分位。考虑到行业基本面复苏，当前的估值具备一定的上行空间。

5. 投资风险：全球经济增长面临压力，服务器、智能手机、新能源车等需求不及预期；半导体行业下行，或细分行业上行动能不足；生成式 AI 需求爆发持续性弱于预期；AI 算力芯片迭代不及预期；行业竞争加剧；投入或费用过大，拖累利润表现。](#)













本研究

## 3.1 LLMImageBlobParser

参数:
- model: llm实例对象。如ChatOllama(model="qwen2.5vl:latest")
- prompt: str, 提示词。默认提示词：

```
"You are an assistant tasked with summarizing images for retrieval. "
    "1. These summaries will be embedded and used to retrieve the raw image. "
    "Give a concise summary of the image that is well optimized for retrieval\n"
    "2. extract all the text from the image. "
    "Do not exclude any content from the page.\n"
    "Format answer in markdown without explanatory text "
    "and without markdown delimiter ``` at the beginning. "
```

翻译过来：

```
你是一名助理，负责对图片进行摘要以便检索。
    “1. 这些摘要将被嵌入并用于检索原始图片。”
    “请提供图片的简洁摘要，并针对检索进行优化\n”
    “2. 从图片中提取所有文本。”
    “请勿从页面中排除任何内容。\n”
    “请将答案格式化为 Markdown 格式，不包含解释性文字”
    “并且开头不包含 Markdown 分隔符 ```。
```

## 3.2 PyMuPDF Pro

pymupdf的收费版。

- PyMuPDF Pro 是针对 PyMuPDF 的一组商业扩展。
- 通过支持 Office 文档和集成 RAG / 大语言模型（LLM）来增强 PyMuPDF 的功能。
- 支持处理 Office 文档，包括 doc、docx、hwp、hwpx

| 功能维度       | PyMuPDF (开源版)      | PyMuPDF Pro (商业版)         |
|----------------|-----------------------|------------------------------|
| **支持格式**   | PDF/XPS               | PDF + 全系 Office 文档        |
| **表格识别**   | 基础算法              | 增强算法 + 格式保留           |
| **LLM 优化**   | PyMuPDF4LLM                   | 包含 PyMuPDF4LLM Pro          |
| **文档转换质量** | 标准                  | 出版级                       |
| **批量处理**   | 需自行实现            | 内置高性能批处理              |
| **技术支持**   | 社区支持              | 专业技术支持                 |
| **商业授权**   | AGPL 许可             | 需购买商业授权               |

## 3.3 Artifex Software 公司

公司概况
- 成立时间：1993年
- 总部：美国加利福尼亚州
- 核心业务：文档处理技术和软件授权
  
核心技术产品：

| 产品线      | 技术特点                          | 商业模型               |
|------------|----------------------------------|-----------------------|
| **Ghostscript** | PostScript/PDF 解释器            | 双许可(AGPL/商业)     |
| **MuPDF**   | 轻量级 PDF 渲染引擎               | 开源(AGPL) + 商业扩展 |
| **PyMuPDF** | MuPDF 的 Python 绑定             | 开源 + Pro 商业版     |